In [1]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
import numpy as np

# Use the context manager to redirect stdout and stderr to the null device
class MinuteDataPredictor:

    def __init__(self, symbol, file_path):
        self.symbol = symbol
        self.file_path = file_path
        self.minute_data = None
        self.correct_predictions=0
        self.wrong_predictions=0

    
    def fetch_minute_data(self):
        try:
            # Fetch historical data for the last x amount of days with x amount of minute intervals
            data = yf.download(self.symbol, period='2d', interval='60m')

            # Select only the 'Close' prices and reset the index to include 'Timestamp'
            self.minute_data = data[['Close']].reset_index()

            # Rename columns to match 'Timestamp' and 'Price'
            self.minute_data.rename(columns={'Datetime': 'Timestamp', 'Close': 'Price'}, inplace=True)

            # Format the timestamp column
            self.minute_data['Timestamp'] = self.minute_data['Timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")

            # Add empty columns for 'Prediction' and 'Accuracy'
            self.minute_data['Prediction_Prophet'] = None
            self.minute_data['Accuracy_Prophet'] = None
        
        except Exception as e:
            print(f"Error fetching minute data: {e}")

    def prophet_predictions(self, data):

            data = data[['Timestamp', 'Price']].copy()
            data.rename(columns={'Timestamp': 'ds', 'Price': 'y'}, inplace=True)
            model = Prophet()
            model.fit(data)
            future = model.make_future_dataframe(periods=1, freq='min')
            forecast = model.predict(future)
            next_minute_prediction = forecast['yhat'].iloc[-1]
            #print(next_minute_prediction)
            return next_minute_prediction

# Set the logging level to suppress informational messages

        
    def save_minute_data_with_predictions(self):
        try:
            self.fetch_minute_data()

            for i in range(1, len(self.minute_data) - 1):
                previous_data = self.minute_data.iloc[:i + 1].copy()
                
                # Make predictions using Prophet model
                next_minute_prediction_prophet = self.prophet_predictions(previous_data)
                self.minute_data.at[i, 'Prediction_Prophet'] = next_minute_prediction_prophet
                
                
                actual_price = self.minute_data.loc[i, 'Price']
                actual_change = self.minute_data.loc[i + 1, 'Price'] - actual_price
                predicted_change = next_minute_prediction_prophet - actual_price
                
                if (predicted_change > 0 and actual_change > 0 ):
                    self.correct_predictions += 1

                elif (predicted_change < 0 and actual_change < 0 ):
                    self.correct_predictions += 1
                    
                else:
                    self.wrong_predictions+=1
                total = self.correct_predictions+self.wrong_predictions
                if total !=0:
                    
                    accuracy = (self.correct_predictions / total) * 100
                else:
                    accuracy = 0

                self.minute_data.at[i, 'Accuracy_Prophet'] = accuracy
                                   
            self.minute_data.to_csv(self.file_path, index=False)
           # print(f"Minute data with predictions and accuracy based on model saved to {self.file_path}")

        except Exception as e:
            print(f"Error: {e}")
predictor = MinuteDataPredictor(symbol='AAPL', file_path='newminute_data_aapl_with_prediction_oop.csv')

predictor.save_minute_data_with_predictions()
Stock = pd.read_csv('newminute_data_aapl_with_prediction_oop.csv',  index_col=0)
df_Stock = Stock
df_Stock.head(10)

[*********************100%%**********************]  1 of 1 completed


20:59:06 - cmdstanpy - INFO - Chain [1] start processing
20:59:07 - cmdstanpy - INFO - Chain [1] done processing
20:59:07 - cmdstanpy - INFO - Chain [1] start processing
20:59:09 - cmdstanpy - INFO - Chain [1] done processing
20:59:09 - cmdstanpy - INFO - Chain [1] start processing
20:59:15 - cmdstanpy - INFO - Chain [1] done processing
20:59:15 - cmdstanpy - INFO - Chain [1] start processing
20:59:21 - cmdstanpy - INFO - Chain [1] done processing
20:59:22 - cmdstanpy - INFO - Chain [1] start processing
20:59:22 - cmdstanpy - INFO - Chain [1] done processing
20:59:22 - cmdstanpy - INFO - Chain [1] start processing
20:59:22 - cmdstanpy - INFO - Chain [1] done processing
20:59:22 - cmdstanpy - INFO - Chain [1] start processing
20:59:22 - cmdstanpy - INFO - Chain [1] done processing
20:59:22 - cmdstanpy - INFO - Chain [1] start processing
20:59:22 - cmdstanpy - INFO - Chain [1] done processing
20:59:23 - cmdstanpy - INFO - Chain [1] start processing
20:59:23 - cmdstanpy - INFO - Chain [1]

,Price,Prediction_Prophet,Accuracy_Prophet
Timestamp,,,
2024-02-15 09:30:00,181.880005,NaN,NaN
2024-02-15 10:30:00,181.990005,181.991917,100.000000
2024-02-15 11:30:00,182.024994,182.025492,100.000000
2024-02-15 12:30:00,182.050003,182.050383,66.666667
2024-02-15 13:30:00,181.978897,181.977675,50.000000
2024-02-15 14:30:00,182.380096,182.301264,40.000000
2024-02-15 15:30:00,183.949997,183.752977,50.000000
2024-02-16 09:30:00,182.544998,182.646338,57.142857
2024-02-16 10:30:00,183.149994,182.833972,62.500000
